In [2]:
%matplotlib inline

from pathlib import Path 

import dask.dataframe as dd
import pandas as pd

In [3]:
YEAR = 2019

In [4]:
slookup = pd.read_csv('ghcn_mos_lookup.csv')

# GHCN

In [5]:
names = ['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG', 'OBS-TIME']
ds = dd.read_csv(f's3://noaa-ghcn-pds/csv/{YEAR}.csv', storage_options={'anon':True},  
                 names=names, parse_dates=['DATE'], dtype={'DATA_VALUE':'object'})

ghcn = ds[['ID', 'DATE', 'ELEMENT', 'DATA_VALUE']][ds['ID'].isin(slookup['ID']) & ds['ELEMENT'].str.match('TAVG')].compute()

In [7]:
ghcn.head()

,ID,DATE,ELEMENT,DATA_VALUE
60868,USC00244558,2019-01-01,TAVG,-74
84360,USW00003017,2019-01-01,TAVG,-155
84614,USW00003170,2019-01-01,TAVG,-144
84748,USW00003810,2019-01-01,TAVG,154
84771,USW00003812,2019-01-01,TAVG,145


# MOS

In [8]:
file_list = list(Path(f'mos{YEAR}').iterdir())

In [9]:
mos_tables = pd.DataFrame()
for f in sorted(file_list):
    mos = pd.read_csv(f)[['station', 'runtime','TMP', 'forecast']].drop_duplicates()
    #somehow there's a row where the header names got repeated
    mos.drop(mos[mos['station'].str.match('station')].index, inplace=True)
    #filter & convert
    days, _, hours = mos['forecast'][0].split()
    col_name = f'TMP_{days}:{hours.split(":")[0]}'
    mosc = mos[['station', 'TMP']].astype({'TMP':float})
    mosc['runtime'] = pd.to_datetime(mos['runtime'])
    mos_table = mosc.groupby(['station', pd.Grouper(freq='D', key='runtime')]).mean().reset_index()
    mos_table['date'] = mos_table['runtime'].dt.date
    if len(mos_tables) == 0:
        mos_tables = mos_table[['station','date']]
    else:
        if not(mos_tables['station'].eq(mos_table['station']).all() or mos_tables['date'].eq(mos_table['date']).all()):
            continue
    mos_tables[col_name] = mos_table['TMP']

<ipython-input-9-064848d0325b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mos_tables[col_name] = mos_table['TMP']


In [10]:
mos_tables.columns

Index(['station', 'date', 'TMP_0:06', 'TMP_0:09', 'TMP_0:12', 'TMP_0:15',
       'TMP_0:18', 'TMP_0:21', 'TMP_1:00', 'TMP_1:03', 'TMP_1:06', 'TMP_1:09',
       'TMP_1:12', 'TMP_1:15', 'TMP_1:18', 'TMP_1:21', 'TMP_2:00', 'TMP_2:03',
       'TMP_2:06', 'TMP_2:09', 'TMP_2:12', 'TMP_2:18', 'TMP_3:00'],
      dtype='object')

In [11]:
slook = slookup.rename(columns={'Station':'station'})[['ID', 'station']]
slook.dtypes

ID         object
station    object
dtype: object

In [12]:
mos_tables.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206955 entries, 0 to 206954
Data columns (total 23 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   station   206955 non-null  object 
 1   date      206955 non-null  object 
 2   TMP_0:06  206955 non-null  float64
 3   TMP_0:09  206955 non-null  float64
 4   TMP_0:12  206955 non-null  float64
 5   TMP_0:15  206955 non-null  float64
 6   TMP_0:18  206955 non-null  float64
 7   TMP_0:21  206955 non-null  float64
 8   TMP_1:00  206955 non-null  float64
 9   TMP_1:03  206955 non-null  float64
 10  TMP_1:06  206955 non-null  float64
 11  TMP_1:09  206955 non-null  float64
 12  TMP_1:12  206955 non-null  float64
 13  TMP_1:15  206955 non-null  float64
 14  TMP_1:18  206955 non-null  float64
 15  TMP_1:21  206955 non-null  float64
 16  TMP_2:00  206955 non-null  float64
 17  TMP_2:03  206955 non-null  float64
 18  TMP_2:06  206955 non-null  float64
 19  TMP_2:09  206955 non-null  float64
 20  TMP_

In [13]:
moswn = pd.merge(mos_tables, slook, how='right', on='station')

In [14]:
moswn.head(1)

,station,date,TMP_0:06,TMP_0:09,TMP_0:12,TMP_0:15,TMP_0:18,TMP_0:21,TMP_1:00,TMP_1:03,...,TMP_1:18,TMP_1:21,TMP_2:00,TMP_2:03,TMP_2:06,TMP_2:09,TMP_2:12,TMP_2:18,TMP_3:00,ID
0,K04W,2019-01-01,2.0,2.0,2.25,2.0,2.5,2.5,3.0,4.25,...,13.75,16.25,18.5,20.75,22.75,23.25,24.75,28.0,30.5,USW00054932


In [15]:
moswn['DATE'] = pd.to_datetime(moswn['date'])

In [16]:
pairup = pd.merge(moswn, ghcn, how='inner', on=['ID', 'DATE']).rename(columns={'DATA_VALUE':'TMP'})

In [17]:
pairup.columns

Index(['station', 'date', 'TMP_0:06', 'TMP_0:09', 'TMP_0:12', 'TMP_0:15',
       'TMP_0:18', 'TMP_0:21', 'TMP_1:00', 'TMP_1:03', 'TMP_1:06', 'TMP_1:09',
       'TMP_1:12', 'TMP_1:15', 'TMP_1:18', 'TMP_1:21', 'TMP_2:00', 'TMP_2:03',
       'TMP_2:06', 'TMP_2:09', 'TMP_2:12', 'TMP_2:18', 'TMP_3:00', 'ID',
       'DATE', 'ELEMENT', 'TMP'],
      dtype='object')

In [18]:
temps = pairup[['station', 'ID', 'DATE','TMP','TMP_0:06', 'TMP_0:09', 'TMP_0:12', 'TMP_0:15',
       'TMP_0:18', 'TMP_0:21', 'TMP_1:00', 'TMP_1:03', 'TMP_1:06', 'TMP_1:09',
       'TMP_1:12', 'TMP_1:15', 'TMP_1:18', 'TMP_1:21', 'TMP_2:00', 'TMP_2:03',
       'TMP_2:06', 'TMP_2:09', 'TMP_2:12', 'TMP_2:18', 'TMP_3:00']]

In [19]:
temps.to_csv(f"ALL_{YEAR}.csv")